## Display LSSTCam images
In this notebook, we show how to query the LSSTCam repository\
and view the resulting images.\

Craig Lage - 16-Apr-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.math as afwMath
import lsst.afw.display as afwDisplay
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded'])
camera = LsstCam.getCamera()
instrument = "LSSTCam"

In [ ]:
butler.registry.queryDatasetTypes()

In [ ]:
butler.registry.queryCollections('LSSTCam/runs*')

In [ ]:
butler.query_datasets('visit_image')

In [ ]:
butler.registry.queryDatasetTypes('visit_image')

In [ ]:
dayObs = 20250915
seqNum = 318
detNum = 55
expId = 2025080900334
expId = int(dayObs * 1E5 + seqNum)
calexp = butler.get('visit_image', detector=detNum, exposure=expId, instrument=instrument)


In [ ]:
%matplotlib inline
x = plot(calexp, stretch='ccs')
x

In [ ]:
x = plot(calexp.image.array[1500:2000, 1250:1750], stretch='linear')
x

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])
camera = LsstCam.getCamera()
instrument = "LSSTCam"

In [ ]:
dayObs = 20250905


exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


## Define a simple ISR
### Just overscan subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=False
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=False
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR on one CCD and look at the result

In [ ]:
expId = 2025090500272
detName = 'R24_S11'
det = camera[detName]
detNum = det.getId()
calexp = butler.get('preliminary_visit_image', detector=detNum, visit=expId, instrument=instrument)

In [ ]:
plt.imshow?

In [ ]:
plot?

In [ ]:
%matplotlib inline
x = plot(calexp, stretch='ccs')
axs = x.get_axes()
axs[0].set_title(f"{expId} {detNum}")
x.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_{expId}_{detNum}.png")
x

In [ ]:
x = plot(calexp.image.array[2400:2600, 1700:1900], stretch='ccs')
x

In [ ]:
window_size = 500
weights = np.ones(window_size) / window_size
sma = np.convolve(calexp.image.array[2500,:], weights, mode='valid')
plt.plot(sma)

In [ ]:
test = calexp.image.array - np.median(calexp.image.array)
plt.imshow(test, vmin=0.1, vmax=10, origin='lower', cmap='gray')
plt.axis('off')
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_Imshow_{expId}_{detNum}.png")

In [ ]:
plt.imshow(isrResult.exposure.image.array[1000:2000, 0:1000], vmin=50000, vmax=150000, origin='lower')
plt.colorbar()
#plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_{expId}_{detector}.png")

In [ ]:
med = np.nanmedian(exp.image.array.flatten())
med

In [ ]:
def isrCallback(im, ccd, imageSource):
    """Assemble the CCD image and do basic ISR"""
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('raw', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    ptc = imageSource.butler.get('ptc', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    isrResult = isrTask.run(exp, ptc)
    nQuarter = ccd.getOrientation().getNQuarter()
    oim = afwMath.rotateImageBy90(isrResult.exposure.image, nQuarter)
    return oim

def calexpCallback(im, ccd, imageSource):
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('preliminary_visit_image', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    print(ccd.getId(), np.nanmedian(exp.image.array.flatten()))
    oim = exp.image
    return oim

def rawCallback(im, ccd, imageSource):
    """Assemble the CCD image.  Just bias subtraction and gain correction"""
    print(ccd.getId(), np.nanmedian(exp.image.array.flatten()))
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=True, correctGain=False)
    return oim

In [ ]:
rafts = ['R32','R33','R42','R43']
ccds_lower = ['S00','S01','S02',
        'S10','S11','S12']
ccds_upper = ['S10','S11','S12',
        'S20','S21','S22']

detectorNameList = []
for raft in rafts:
    if raft in ['R42', 'R43']:
        ccds = ccds_lower
    else:
        ccds = ccds_upper
    for ccd in ccds:
        detectorNameList.append(raft+'_'+ccd)
detectorNameList

In [ ]:
rafts = [      'R01','R02','R03', 
         'R10','R11','R12','R13','R14',
         'R20','R21','R22','R23','R24',
         'R30','R31','R32','R33','R34',  
               'R41','R42','R43']
ccds = ['S00','S01','S02',
        'S10','S11','S12',
        'S20','S21','S22']
detectorNameList = []
for raft in rafts:
    for ccd in ccds:
        detectorNameList.append(raft+'_'+ccd)
detectorNameList

In [ ]:
%matplotlib inline
instrument = "LSSTCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min=20, max=200)
disp.setImageColormap("gray_r")
dayObs = 20250711
seqNum = 176
#detectorNameList = ['R43_S11']
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, 
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=calexpCallback,
                                                       background=np.nan),
                              detectorNameList=detectorNameList,
                              binSize=4, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_Calexp_CutDown_{dayObs}_{seqNum}.png")